In [1]:
# DataSens E1_v1 — 05_snapshot_and_readme
# Objectifs: inspection tables, vue join, snapshot versionné, historique
# Prérequis: 04_crud_tests
# Sorties: datasens/versions/ + MAJ README_VERSIONNING.md
# Référence: docs/GUIDE_TECHNIQUE_E1.md
import sqlite3
from datetime import datetime
from pathlib import Path

import pandas as pd

PROJECT_ROOT = Path.cwd().parents[2] if (Path.cwd().name.startswith('datasens_')) else Path.cwd()
DB_PATH = PROJECT_ROOT / "datasens" / "datasens.db"
VERSION_FILE = PROJECT_ROOT / "README_VERSIONNING.md"
conn = sqlite3.connect(DB_PATH)

# Helpers

def sql_df(query, params=None):
    return pd.read_sql_query(query, conn, params=params)

# Inspection tables clés
for t in ["type_donnee", "source", "flux", "document"]:
    print(f"\n🧱 Table : {t}")
    try:
        display(sql_df(f"SELECT * FROM {t} LIMIT 10;"))
    except Exception:
        pass

# Vue chainée
query_sources = """
SELECT
    td.libelle          AS type_donnee,
    s.nom               AS source_nom,
    ROUND(s.fiabilite,2) AS source_fiabilite,
    f.format            AS flux_format,
    f.date_collecte     AS flux_date,
    f.manifest_uri      AS flux_manifest,
    d.titre             AS doc_titre,
    d.date_publication  AS doc_date
FROM type_donnee td
LEFT JOIN source   s ON s.id_type_donnee = td.id_type_donnee
LEFT JOIN flux     f ON f.id_source      = s.id_source
LEFT JOIN document d ON d.id_flux        = f.id_flux
ORDER BY td.libelle, s.nom, f.date_collecte, d.date_publication;
"""
print("\n📊 Vue d'ensemble de la chaîne de sourcing :")
try:
    display(sql_df(query_sources))
except Exception:
    pass

# Snapshot versionné
BACKUP_DIR = PROJECT_ROOT / "datasens" / "versions"
BACKUP_DIR.mkdir(parents=True, exist_ok=True)

def save_database_version(note="Validation"):
    import shutil
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_name = f"datasens_v{timestamp}.db"
    backup_path = BACKUP_DIR / backup_name
    shutil.copy(DB_PATH, backup_path)
    with open(VERSION_FILE, "a", encoding="utf-8") as f:
        f.write(f"- **{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}** | `DB_BACKUP` | {backup_name} - {note}\n")
    print(f"📦 Snapshot créé : {backup_name}")

save_database_version("Après validation de la chaîne Type-Source-Flux-Document")

# Historique
print("\n📘 Historique des versions DataSens :\n")
try:
    with open(VERSION_FILE, encoding="utf-8") as f:
        print(f.read())
except UnicodeDecodeError:
    with open(VERSION_FILE, encoding="cp1252") as f:
        print(f.read())

print("\n🏁 DataSens E1 terminé avec succès !")



🧱 Table : type_donnee

🧱 Table : source

🧱 Table : flux

🧱 Table : document

📊 Vue d'ensemble de la chaîne de sourcing :
📦 Snapshot créé : datasens_v20251101_123610.db

📘 Historique des versions DataSens :

- **2025-10-30 18:54:51** | `DB_SCHEMA` | Création des 18 tables principales
- **2025-10-30 18:54:56** | `DB_SCHEMA` | Création des 18 tables principales
- **2025-10-30 18:55:22** | `DB_SCHEMA` | Création des 18 tables principales
- **2025-10-30 18:55:46** | `DB_SCHEMA` | Création des 18 tables principales
- **2025-10-30 18:56:12** | `DB_SCHEMA` | Création des 18 tables principales
- **2025-10-30 18:56:42** | `DB_SEED` | Insertion du jeu de données minimal
- **2025-10-30 18:56:47** | `DB_SEED` | Insertion du jeu de données minimal
- **2025-10-30 18:57:08** | `DB_SEED` | Insertion du jeu de données minimal
- **2025-10-30 18:57:29** | `DB_SEED` | Insertion du jeu de données minimal
- **2025-10-30 18:57:50** | `DB_SEED` | Insertion du jeu de données minimal
- **2025-10-30 18:59:47** |

> Notes:
> - Aperçu des tables clés (`type_donnee`, `source`, `flux`, `document`).
> - La vue joinée illustre la traçabilité: Type → Source → Flux → Document.
> - `save_database_version` crée un snapshot horodaté de la base.
> - Affichage de l’historique complet pour audit.

